<font size="+3"><strong>2.5. Predicting Apartment Prices in Mexico City 🇲🇽</strong></font>

In [ ]:
# Import libraries here
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import seaborn as sns
from category_encoders import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge  # noqa F401
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
import plotly.express as px

# Prepare Data

## Import

**Task 2.5.1:** Write a `wrangle` function that takes the name of a CSV file as input and returns a DataFrame. The function should do the following steps:

1. Subset the data in the CSV file and return only apartments in Mexico City (`"Distrito Federal"`) that cost less than \$100,000.
2. Remove outliers by trimming the bottom and top 10\% of properties in terms of `"surface_covered_in_m2"`.
3. Create separate `"lat"` and `"lon"` columns.
4. Mexico City is divided into [15 boroughs](https://en.wikipedia.org/wiki/Boroughs_of_Mexico_City). Create a `"borough"` feature from the `"place_with_parent_names"` column.
5. Drop columns that are more than 50\% null values.
6. Drop columns containing low- or high-cardinality categorical values. 
7. Drop any columns that would constitute leakage for the target `"price_aprox_usd"`.
8. Drop any columns that would create issues of multicollinearity. 

<div class="alert alert-block alert-info">
<b>Tip:</b> Don't try to satisfy all the criteria in the first version of your <code>wrangle</code> function. Instead, work iteratively. Start with the first criteria, test it out with one of the Mexico CSV files in the <code>data/</code> directory, and submit it to the grader for feedback. Then add the next criteria.</div>

In [ ]:
# Build your `wrangle` function
def wrangle(filepath):
    df = pd.read_csv(filepath)
    # subset the data and return only aprtments in mexico city "Distrito Federal"
    
    mask_ba = df["place_with_parent_names"].str.contains("Distrito Federal")
    # subset only the apartment
    
    mask_apt = (df["property_type"]  == "apartment")
    
    # subset the apartments that cost less than $100,00
    
    mask_price = df["price_aprox_usd"] < 100000
    df = df[mask_ba & mask_apt & mask_price]
    
    # Remove outliers by trimming the bottom and top 10% of properties in terms of "surface_covered_in_m2"
    
    low,high = df["surface_covered_in_m2"].quantile([0.1,0.9])
    mask_area = df['surface_covered_in_m2'].between(low,high)
    df = df[mask_area]
    
    #Create separate "lat" and "lon" columns.
    
    df[["lat","lon"]]=df["lat-lon"].str.split("," , expand = True).astype(float)
    df.drop(columns = ["lat-lon"], inplace = True)
    
    # Mexico City is divided into 15 boroughs. Create a "borough" feature from the "place_with_parent_names" column.
    
    df["borough"]=df["place_with_parent_names"].str.split("|" , expand = True)[1]
    df.drop(columns = ["place_with_parent_names"],inplace = True)
    
    #Drop columns that are more than 50% null values.
    
    df.drop(columns = ["price_usd_per_m2", 
    "floor",
    "rooms",
    "expenses",
    "surface_total_in_m2" ],inplace = True)
    
    #Drop columns containing low- or high-cardinality categorical values.
    
    df.drop(columns = ["operation","property_type","currency","properati_url"],inplace = True)
    
    # Drop any columns that would constitute leakage for the target "price_aprox_usd".
    
    df.drop(columns = ["price_aprox_local_currency","price","price_per_m2"], inplace = True)
    return df

In [ ]:
# Use this cell to test your wrangle function and explore the data
df = wrangle("data/mexico-city-real-estate-1.csv")
df.shape

In [ ]:
corr = df.select_dtypes("number").drop(columns = "price_aprox_usd").corr()
sns.heatmap(corr)

**Task 2.5.2:** Use glob to create the list `files`. It should contain the filenames of all the Mexico City real estate CSVs in the `./data` directory, except for `mexico-city-test-features.csv`.

In [ ]:
files = glob("data/mexico-city-real-estate-*.csv")
files


**Task 2.5.3:** Combine your `wrangle` function, a list comprehension, and `pd.concat` to create a DataFrame `df`. It should contain all the properties from the five CSVs in `files`. 

In [ ]:
frames = [wrangle(file) for file in files]
df = pd.concat(frames)

## Explore

**Task 2.5.4:** Create a histogram showing the distribution of apartment prices (`"price_aprox_usd"`) in `df`. Be sure to label the x-axis `"Price [$]"`, the y-axis `"Count"`, and give it the title `"Distribution of Apartment Prices"`. Use Matplotlib (`plt`).

What does the distribution of price look like? Is the data normal, a little skewed, or very skewed?

In [ ]:
skewness = df['price_aprox_usd'].skew()
print(skewness)


In [ ]:
# Build histogram
plt.hist(df["price_aprox_usd"])


# Label axes

plt.xlabel("Price [$]")
plt.ylabel("count")
# Add title
plt.title("Distribution of Apartment Prices")

# Don't delete the code below 👇
plt.savefig("images/2-5-4.png", dpi=150)


**Task 2.5.5:** Create a scatter plot that shows apartment price (`"price_aprox_usd"`) as a function of apartment size (`"surface_covered_in_m2"`). Be sure to label your x-axis `"Area [sq meters]"` and y-axis `"Price [USD]"`. Your plot should have the title `"Mexico City: Price vs. Area"`. Use Matplotlib (`plt`).

In [ ]:
# Build scatter plot
plt.scatter(x = df["surface_covered_in_m2"] , y = df["price_aprox_usd"] )


# Label axes
plt.xlabel("Area [sq meters]")
plt.ylabel("Price [USD]")
# Add title
plt.title("Mexico City: Price vs. Area")

# Don't delete the code below 👇
plt.savefig("images/2-5-5.png", dpi=150)


**Task 2.5.6:** **(UNGRADED)** Create a Mapbox scatter plot that shows the location of the apartments in your dataset and represent their price using color. 

What areas of the city seem to have higher real estate prices?

In [ ]:
import plotly.express as px

# Create a scatter mapbox plot
fig = px.scatter_mapbox(
    df,
    lat='lat',
    lon='lon',
    color='price_aprox_usd',  # Color by price
    color_continuous_scale=px.colors.sequential.Plasma,  # Choose a color scale
    size='price_aprox_usd',  # Size points based on price (optional)
    size_max=10,
    height = 600,# Max size of points
    zoom=12,  # Adjust the zoom level
    mapbox_style='carto-positron'  # Choose a Mapbox style
)

# Set your Mapbox access token
fig.update_layout(mapbox_accesstoken='your_mapbox_access_token')

# Show the map
fig.show()


## Split

**Task 2.5.7:** Create your feature matrix `X_train` and target vector `y_train`. Your target is `"price_aprox_usd"`. Your features should be all the columns that remain in the DataFrame you cleaned above.

In [ ]:
# Split data into feature matrix `X_train` and target vector `y_train`.
features = ['surface_covered_in_m2', 'lat', 'lon', 'borough']
target = "price_aprox_usd"
X_train = df[features]
y_train = df[target]

# Build Model

## Baseline

**Task 2.5.8:** Calculate the baseline mean absolute error for your model.

In [ ]:
y_mean = y_train.mean()
y_pred_baseline = [y_mean]*len(y_train)
baseline_mae = mean_absolute_error(y_train,y_pred_baseline)
print("Mean apt price:", y_mean)
print("Baseline MAE:", baseline_mae)

## Iterate

**Task 2.5.9:** Create a pipeline named `model` that contains all the transformers necessary for this dataset and one of the predictors you've used during this project. Then fit your model to the training data.

In [ ]:
# Build Model
model = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    SimpleImputer(),
    Ridge()
)
model.fit(X_train,y_train)
# Fit model


## Evaluate

**Task 2.5.10:** Read the CSV file `mexico-city-test-features.csv` into the DataFrame `X_test`.

In [ ]:
X_test = pd.read_csv("data/mexico-city-test-features.csv")
print(X_test.info())
X_test.head()

**Task 2.5.11:** Use your model to generate a Series of predictions for `X_test`. When you submit your predictions to the grader, it will calculate the mean absolute error for your model.

In [ ]:
y_test_pred = pd.Series(model.predict(X_test))
y_test_pred.head()

# Communicate Results

**Task 2.5.12:** Create a Series named `feat_imp`. The index should contain the names of all the features your model considers when making predictions; the values should be the coefficient values associated with each feature. The Series should be sorted ascending by absolute value.  

In [ ]:
features = X_train.columns  # Get feature names from DataFrame

# Check the lengths
print("Number of coefficients:", len(coefficients))
print("Number of features:", len(features))

In [ ]:
coefficients = model.named_steps["ridge"].coef_

intercept = model.named_steps["ridge"].intercept_
features = model.named_steps["onehotencoder"].get_feature_names()

feat_imp =  pd.Series(coefficients, index=features)
feat_imp

**Task 2.5.13:** Create a horizontal bar chart that shows the **10 most influential** coefficients for your model. Be sure to label your x- and y-axis `"Importance [USD]"` and `"Feature"`, respectively, and give your chart the title `"Feature Importances for Apartment Price"`. Use pandas. 

In [ ]:
top_features = feat_imp.abs().nlargest(10)
top_features

In [ ]:
# Build bar chart

top_features.plot(kind='barh', color='skyblue', figsize=(10, 6))


#op_features = feat_imp.abs().nlargest(10)

# Build bar chart
top_features.plot(kind='barh', color='skyblue', figsize=(10, 6))

# Label axes
plt.xlabel('Importance [USD]')
plt.ylabel('Feature')

# Add title
plt.title('Feature Importances for Apartment Price')

# Show the plot
plt.show()

# Don't delete the code below 👇
plt.savefig("images/2-5-13.png", dpi=150)